In [2]:
import os
import cv2
import numpy as np
import xml.etree.ElementTree as ET
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from sklearn.model_selection import train_test_split

In [3]:
data_dir = "C:/Users/DELL/null_class/traffic_model/vehicle"

Parsing XML annotations

In [4]:
def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    boxes = []
    labels = []
    
    for obj in root.findall('object'):
        label = obj.find('name').text
        xmin = int(obj.find('bndbox/xmin').text)
        ymin = int(obj.find('bndbox/ymin').text)
        xmax = int(obj.find('bndbox/xmax').text)
        ymax = int(obj.find('bndbox/ymax').text)
        boxes.append((xmin, ymin, xmax, ymax))
        labels.append(label)
    
    return boxes, labels


In [5]:
image_paths = []
boxes_list = []
labels_list = []

In [6]:
image_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.jpg')])
for image_file in image_files:
    image_path = os.path.join(data_dir, image_file)
    xml_file = os.path.join(data_dir, os.path.splitext(image_file)[0] + '.xml')
    
    if not os.path.exists(xml_file):
        continue
    
    boxes, labels = parse_xml(xml_file)
    image_paths.append(image_path)
    boxes_list.append(boxes)
    labels_list.append(labels)

Load and preprocess images

In [7]:
def load_and_preprocess_image(image_path, boxes):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image = cv2.resize(image, (224, 224)) 
    image = image / 255.0  
    return image

In [8]:
images = []
for i, image_path in enumerate(image_paths):
    boxes = boxes_list[i]
    image = load_and_preprocess_image(image_path, boxes)
    images.append(image)

In [9]:
car_counts = []
other_vehicle_counts = []

In [10]:
for labels in labels_list:
    car_count = sum(1 for label in labels if label == 'car')
    other_vehicle_count = sum(1 for label in labels if label != 'car')
    car_counts.append(car_count)
    other_vehicle_counts.append(other_vehicle_count)

In [11]:
images = np.array(images)
car_counts = np.array(car_counts)
other_vehicle_counts = np.array(other_vehicle_counts)

Splitting the data

In [12]:
train_images, val_images, train_car_counts, val_car_counts, train_other_vehicle_counts, val_other_vehicle_counts = train_test_split(
    images, car_counts, other_vehicle_counts, test_size=0.2, random_state=42)


In [13]:
print(f'Train images shape: {train_images.shape}')
print(f'Train car counts shape: {train_car_counts.shape}')
print(f'Train other vehicle counts shape: {train_other_vehicle_counts.shape}')
print(f'Validation images shape: {val_images.shape}')
print(f'Validation car counts shape: {val_car_counts.shape}')
print(f'Validation other vehicle counts shape: {val_other_vehicle_counts.shape}')

Train images shape: (2220, 224, 224, 3)
Train car counts shape: (2220,)
Train other vehicle counts shape: (2220,)
Validation images shape: (555, 224, 224, 3)
Validation car counts shape: (555,)
Validation other vehicle counts shape: (555,)


Define the model

In [14]:
model = Sequential([
    Input(shape=(224, 224, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(2)  
])

In [15]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     5,537,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,631,234 (21.48 MB)

 Trainable params: 5,631,234 (21.48 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
train_labels = np.column_stack((train_car_counts, train_other_vehicle_counts))
val_labels = np.column_stack((val_car_counts, val_other_vehicle_counts))

In [18]:
model.fit(train_images, train_labels, epochs=50, validation_data=(val_images, val_labels))

Epoch 1/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 86s 1s/step - loss: 41.5097 - mae: 3.1239 - val_loss: 24.9307 - val_mae: 2.6554
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - loss: 30.4537 - mae: 2.9738 - val_loss: 22.4205 - val_mae: 2.8296
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 94s 1s/step - loss: 31.4621 - mae: 2.9272 - val_loss: 27.9698 - val_mae: 3.5486
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - loss: 28.6471 - mae: 2.8971 - val_loss: 18.4287 - val_mae: 2.3579
Epoch 5/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 72s 1s/step - loss: 21.1567 - mae: 2.4150 - val_loss: 22.4709 - val_mae: 2.3902
Epoch 6/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1450s 21s/step - loss: 20.2285 - mae: 2.3004 - val_loss: 16.9800 - val_mae: 2.2343
Epoch 7/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 75s 1s/step - loss: 13.1343 - mae: 2.1027 - val_loss: 22.1811 - val_mae: 2.3734
Epoch 8/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - loss: 10.7530 - mae: 1.9777 - val_loss: 16.4657 - val_mae: 2.3806
Epoch 9/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - l

In [19]:
model.save('Vehicle_Count_Model.keras') 